In [1]:
import pandas as pd
import xlrd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [9]:
df1 = pd.read_excel('Central Branch  trackers/Central Division 2021 Debris Removal Tracker.xlsx', parse_dates=True)
df2 = pd.read_excel('tetra forms ASB and SA/Site_assessment tracker.xlsx', parse_dates=True)

In [3]:
df3 =pd.read_excel("tetra forms ASB and SA/SA_TetraForms1.xlsx", parse_dates=True)

In [10]:
smartsheets = df1.copy()
sa = df2.copy()
tetarforms = df3.copy()

In [12]:
smartsheets[(smartsheets['Site Assessment'].notnull()) & (smartsheets['RAD_MERC Sweep'].isnull())].to_excel('No sweep dates.xlsx')

In [7]:
smartsheets.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 645 entries, 0 to 644
Data columns (total 135 columns):
 #    Column                                              Dtype         
---   ------                                              -----         
 0    APN_ROW Segment                                     object        
 1    ROE Type                                            object        
 2    Structural Status                                   object        
 3    Haz Tree Status                                     object        
 4    Debris Crew                                         object        
 5    Tree Crew                                           float64       
 6    Street #                                            object        
 7    Street Name                                         object        
 8    Unit #                                              object        
 9    City                                                object        
 10   County      

# Site Assessment site up

In [5]:
# sa column selection
sa = sa[['APN','Date Completed','Automobiles', 'Motorcycle', 'ATV/UTV/Snowmobiles',
       'Ag/Commercial/Construction', 'RV', 'Trailers', 'Vessel', 'Chimney','Division']]

In [6]:
# only use central division APNs and parcel information
sa = sa[sa['Division'].isin(['Central Division'])]

In [7]:
# replace all 0s to No and all 1 to Yes for Chimney to match smartsheets
sa['Chimney'] = sa['Chimney'].fillna(0).astype(int).astype(str).str.replace("0", "No").str.replace("1", "Yes")


In [8]:
# set up date columns
sa['Date Completed'] = pd.to_datetime(sa['Date Completed']).dt.date

In [9]:
sa.rename(columns={'Date Completed': 'SA Survey Date','APN': 'APN_SA','Motorcycle':'Motorcycle_SA',
                  'RV': 'RV_SA', 'Trailers':'Trailers_SA', 'Vessel': 'Vessel_SA'}, inplace=True)

In [10]:
# set up numerical values for al vehicles
sa['Automobiles'] = pd.to_numeric(sa['Automobiles'])
sa['Motorcycle_SA'] = pd.to_numeric(sa['Motorcycle_SA'])
sa['ATV/UTV/Snowmobiles'] = pd.to_numeric(sa['ATV/UTV/Snowmobiles'])
sa['Ag/Commercial/Construction'] = pd.to_numeric(sa['Ag/Commercial/Construction'])
sa['RV_SA'] = pd.to_numeric(sa['RV_SA'])
sa['Trailers_SA'] = pd.to_numeric(sa['Trailers_SA'])
sa['Vessel_SA'] = pd.to_numeric(sa['Vessel_SA'])

In [11]:
sa.fillna({'Automobiles': 0,
           'Motorcycle_SA': 0,
           'ATV/UTV/Snowmobiles': 0,
           'Ag/Commercial/Construction': 0,
           'RV_SA': 0,
           'Trailers_SA': 0,
           'Vessel_SA': 0}, inplace=True)

In [12]:
sa.columns = map(str.upper, sa.columns)

# Tetra Forms setup

In [13]:
# get just the counties we need
tetarforms = tetarforms[tetarforms['county'].isin(['ALPINE', 'EL DORADO', 'LAKE', 'NEVADA', 'PLACER'])]

In [14]:
# format dates 
tetarforms['survey_date'] = pd.to_datetime(tetarforms['survey_date']).dt.date

In [15]:
#get columsn we need
tetarforms = tetarforms[['apn','survey_date','chimney_quantity', 'automobile_quantity', 'motorcycle_quantity',
                         'atv_utv_quantity', 'comm_equip_quantity', 'rvs_quantity', 'trailer_quantity', 'vessel_quantity']]

In [16]:
tetarforms.fillna({'automobile_quantity': 0,
           'motorcycle_quantity': 0,
           'atv_utv_quantity': 0,
           'comm_equip_quantity': 0,
           'rvs_quantity': 0,
           'trailer_quantity': 0,
           'vessel_quantity': 0}, inplace=True)

In [17]:
tetarforms['chimney_quantity'] = np.where(tetarforms['chimney_quantity'] > 0, 'Yes', 'No')

In [18]:
tetarforms.rename(columns={'apn': 'APN_TF', 'motorcycle_quantity':'motorcycle_quantity_TF'}, inplace=True)

In [19]:
tetarforms.columns = map(str.upper, tetarforms.columns)

In [20]:
# merge1 tetraforms and SA
merge1 = sa.merge(tetarforms,
                 left_on='APN_SA',
                 right_on='APN_TF',
                 how='outer',
                 suffixes=('_SA', '_TF'))

# Smart sheet set up

In [21]:
# smartsheets.info(verbose=True, show_counts=True)

In [23]:
# smart sheet column selection
smartsheets = smartsheets[['APN_ROW Segment','Street #','Street Name', 'Structural Status', 'Chimney', 'Site Assessment',
                          'Number of Passenger Vehicles', 'Number of ATV_UTV_snowmobiles',
                          'Number of Ag_Commercial_Construction Equipment','Number of Motorcycles',
                          'Number of RV\'s', 'Number of Trailers', 'Number of Vessels (Boat_Jetski)', 'Total Number of Vehicles']]

In [25]:
smartsheets = smartsheets.copy()

In [26]:
smartsheets.fillna({'Chimney': "No",
            'Number of Passenger Vehicles': 0,
           'Number of ATV_UTV_snowmobiles': 0,
           'Number of Ag_Commercial_Construction Equipment': 0,
           'Number of Motorcycles': 0,
            'Number of RV\'s':0,
           'Number of Trailers': 0,
           'Number of Vessels (Boat_Jetski)': 0}, inplace=True)

In [27]:
# take of the time on the dates
smartsheets['Site Assessment'] = smartsheets['Site Assessment'].dt.date

In [28]:
smartsheets.rename(columns={'Site Assessment': 'SA Survey Date', 'Number of Passenger Vehicles': 'Automobiles',
                           'Number of ATV_UTV_snowmobiles': 'ATV/UTV/Snowmobiles',
                           'Number of Ag_Commercial_Construction Equipment': 'Ag/Commercial/Construction',
                           'Number of Motorcycles': 'Motorcycles_SS', 'Number of RV\'s': 'RV_SS',
                           'Number of Trailers':'Trailers_SS','Number of Vessels (Boat_Jetski)':'Vessel_SS'}, inplace=True)

In [29]:
smartsheets.columns = map(str.upper, smartsheets.columns)

In [30]:
# merge both data frames for QC
final = smartsheets.merge(merge1,
                         left_on='APN_ROW SEGMENT',
                         right_on='APN_SA',
                         how='outer',
                         suffixes=('_SS', '_SA'))

In [31]:
# set Smart sheets apn as index
final.set_index('APN_ROW SEGMENT', inplace=True)

In [32]:
# final.reset_index(inplace=True)

In [33]:
# final.columns

In [34]:
# column orinizationa
final = final[['APN_SA', 'APN_TF','STREET #','STREET NAME','STRUCTURAL STATUS',
               'SA SURVEY DATE_SS', 'SA SURVEY DATE_SA','SURVEY_DATE',
               'CHIMNEY_SS','CHIMNEY_SA','CHIMNEY_QUANTITY',
               'AUTOMOBILES_SS', 'AUTOMOBILES_SA','AUTOMOBILE_QUANTITY',
               'MOTORCYCLES_SS', 'MOTORCYCLE_SA', 'MOTORCYCLE_QUANTITY_TF',
              'ATV/UTV/SNOWMOBILES_SS', 'ATV/UTV/SNOWMOBILES_SA','ATV_UTV_QUANTITY',
              'AG/COMMERCIAL/CONSTRUCTION_SS', 'AG/COMMERCIAL/CONSTRUCTION_SA','COMM_EQUIP_QUANTITY',
              'RV_SS', 'RV_SA','RVS_QUANTITY',
              'TRAILERS_SS', 'TRAILERS_SA','TRAILER_QUANTITY',
              'VESSEL_SS','VESSEL_SA','VESSEL_QUANTITY',
               'DIVISION','TOTAL NUMBER OF VEHICLES']]

In [35]:
final.to_excel('Central Dixie Fire SA Audit.xlsx')